In [1]:
import spacy
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
import matplotlib as mpl
from _get_adj_phrases import get_adjective_phrases, get_list_of_phrases
from collections import Counter, defaultdict
import math

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
reviews = pd.read_json('../data/reviewSelected100.json', encoding='ISO-8859-1', lines=True)

A sample of what we get from real life data

In [4]:
# get reviews for a random business
random_business = reviews.sample(random_state=42)
random_business_id = random_business.iloc[0]['business_id']
small_business_dataset = reviews.loc[reviews['business_id'] == random_business_id]
small_business_dataset.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
11200,wysFaMq5S88mF6HLxdh3Vw,4nQ7MOkbc5u-DYUOgjiDsg,yEZn1XpLsEC9uBa-X4xAZw,4,1,0,0,Cute little vietnamese sandwich place on St De...,2016-03-17 14:29:44
11204,z81YK3CXQJb48BU7ZAcf7A,Ht8iGitRu8kynEubcIhsTQ,yEZn1XpLsEC9uBa-X4xAZw,4,7,2,8,"Hey Hey, it's the SAMMIES!\n\nSuper Yummy here...",2011-11-30 03:26:01
11209,tqajcnZA97HlXkIYrdNVlg,-a0XgJCXoJln2Ue_JMnfIQ,yEZn1XpLsEC9uBa-X4xAZw,3,0,0,0,Sandwich was good but service was terrible. We...,2016-07-14 18:52:34
11212,wXz1c5kz0QB2nvZVTnnkIg,F9ivL7-mzKbfnLO1rKUFFQ,yEZn1XpLsEC9uBa-X4xAZw,5,0,0,0,"Super fresh, delicious and cheap Banh Mi in th...",2017-07-28 17:48:01
11258,89J69mHgTTF25Ln97ONTrQ,La1ggB37gvq6UWpEWaMtMA,yEZn1XpLsEC9uBa-X4xAZw,4,2,0,2,Que ce soit pour leurs délicieux bubbles tea/s...,2012-12-04 21:25:13


In [5]:
small_business_dataset_reviews = list(small_business_dataset['text'])
# lowercase and correct the text
# this probably works but it takes too long to run so just run this when we want to submit it
# small_business_dataset_reviews = [TextBlob(text).correct() for text in small_business_dataset_reviews]
small_business_dataset_reviews = [sent_tokenize(text) for text in small_business_dataset_reviews]
small_business_dataset_reviews = [item for sublist in small_business_dataset_reviews for item in sublist]

In [6]:
small_business_phrases = [get_adjective_phrases(text.lower()) for text in small_business_dataset_reviews]
small_business_phrases = [array for array in small_business_phrases if len(array) > 0]
small_business_phrases = [text for sublist in small_business_phrases for text in sublist]
small_business_phrases

['really big',
 'very reasonable',
 'here super yummy',
 'really good',
 'very clear',
 'super fresh',
 'so good',
 'so fresh',
 'so good',
 'so cheap',
 'so good',
 'quite hungry',
 'very little',
 'too way sweet',
 'as just good',
 'how delicious',
 'so addictive',
 'really disappointing',
 'very sad',
 'tremendously tasty',
 'so good',
 'pretty cheap',
 'quite cheap',
 'too lazy',
 'very healthy',
 'hard pressed',
 'as good',
 'how friendly',
 'as good',
 'milky iced',
 'pretty small',
 'really excited',
 'so terrible',
 'very reasonable',
 'quite tasty',
 'very tender',
 'too many',
 'already pre already pre already pre already pre already pre already pre ',
 'cloyingly sweet',
 'completely foreign',
 'more western',
 'less strong',
 'too salty',
 'so delicious',
 'absolutely delicious',
 'so good',
 'most familiar',
 'very unusual',
 'definitely best',
 'real good',
 'very enjoyable',
 'very good',
 'very good',
 'more flavorful',
 'too bad',
 'just average',
 'just average',
 'so

In [7]:
phrase_counts = Counter(small_business_phrases)
freq_phrases = phrase_counts.most_common(10)
freq_phrases


[('so good', 8),
 ('very good', 5),
 ('as good', 3),
 ('too sweet', 3),
 ('very reasonable', 2),
 ('super fresh', 2),
 ('pretty cheap', 2),
 ('hard pressed', 2),
 ('just average', 2),
 ('much better', 2)]

## Doing it for all of the data that we have

In [8]:
reviews

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8aoJJdKEO3ypoZNszpPu7Q,bGgAL09pxLnV_FFgR4ZADg,ZBE-H_aUlicix_9vUGQPIQ,5,0,0,0,We had my Mother's Birthday Party here on 10/2...,2016-11-09 20:07:25
1,J5NOCLdhuhor7USRhtYZ8w,pFCb-1j6oI3TDjr26h2cJQ,e-YnECeZNt8ngm0tu4X9mQ,4,0,0,0,Good Korean grill near Eaton Centre. The marin...,2015-12-05 05:06:43
2,PXiLWAYRt3xnHaJ8MB4rzw,mEzc6LeTNiQgIVsq3poMbg,j7HO1YeMQGYo3KibMXZ5vg,5,2,1,3,Was recommended to try this place by few peopl...,2014-10-11 05:16:15
3,VrLarvxZYJm74yAqtpe9PQ,o-zUN2WEZgjQS7jnNsec0g,7e3PZzUpG5FYOTGt3O3ePA,3,0,0,0,Ambience: Would not expect something this nice...,2016-07-25 03:45:26
4,C1CUpidlVFprUCkApqzCmA,Wlx0iBXJvk4x0EeOt2Bz1Q,vuHzLZ7nAeT-EiecOkS5Og,1,11,0,3,Absolutely the WORST pool company that I have ...,2016-04-11 18:49:11
...,...,...,...,...,...,...,...,...,...
15295,qknwFVEh_0KSuexigYBI_A,YMGmyPOU65SMs4H60ltYiw,shIPnFoXrL3dFo5HLH1_HA,1,2,0,0,This was the worst experience ever. So much so...,2014-07-12 21:58:15
15296,Y-ZRoyAXCukBK1uK1ZcZCA,JLhOWQiWtGbr14K_KmoWxA,zPEYgVqJ2QNKi45FJi2jvg,5,0,0,0,We come here every time we hit Vegas! A giant ...,2018-11-10 21:38:49
15297,A8HdjBfhj3pgQuSbwNtDEw,6CoiKFDFXIACJZvv_I_8mQ,zPEYgVqJ2QNKi45FJi2jvg,1,0,1,0,As locals we used to the this place when it w...,2018-10-13 22:11:22
15298,2n1QdrYBRAAe6GKaxEV0jA,_fH4s3ls08eSl_PfX38KIA,etzDsNjkCyQBoJcU2a3U-g,5,0,0,0,The food was delicious. We were seated in 15 m...,2015-02-15 08:43:46


In [9]:
reviews_list = reviews[['business_id', 'text']]
reviews_list = reviews_list.groupby(['business_id'])['text'].apply(','.join).reset_index()
reviews_list

,business_id,text
0,--I7YYLada0tSLkORTHb5Q,"Had to get my wing fix, I like dry rubs on win..."
1,-7XWJYkutqhIxLen7Grg1g,Definite recommend. \n\nBut I never would have...
2,0Rni7ocMC_Lg2UH0lDeKMQ,We love Barros!! Usually go to other locations...
3,0kPm1zEpeXFRg8D2phqgCQ,"Coffee is exponentially better than Starbucks,..."
4,1Fpk8ibHhZYnCw8fnGny8w,Really love the food here! I was a HUGE fan of...
...,...,...
148,yHHVKa9joZAKiBDUp2SkKw,"This coffee shop has great coffee, but more im..."
149,z8Em-bhZI3Mmspml7tj6tg,This was the first time and probably the last ...
150,zPEYgVqJ2QNKi45FJi2jvg,Stopped by here for lunch yesterday. What a d...
151,zZ7KDK3GAkBUZzsaqB1A4Q,"Staff is SO friendly, I was checked in on once..."


In [10]:
# reviews_dict: list of dicts, one for each business
reviews_dict = reviews_list.to_dict('records')

In [11]:
#trial code
reviews_test = reviews_dict[:2]
test = [get_list_of_phrases(review['text']) for review in reviews_test]
test[0]

Counter({'always good always good always good ': 1,
         'just typical': 1,
         'just average': 1,
         'also fantastic also fantastic also fantastic also fantastic ': 1,
         'how good': 1,
         'how delicious': 1,
         'very pleased': 2,
         'very good': 8,
         'very friendly': 4,
         'really good': 7,
         'very prompt': 1,
         'real good': 1,
         'too much': 2,
         'pretty decent': 1,
         'pretty good': 9,
         'most recent': 1,
         'pretty average': 1,
         'way friendlier': 1,
         'exceptionally good': 1,
         'much better': 2,
         'extremely greasy': 1,
         'very empty': 1,
         'enough large': 1,
         'barely hot': 1,
         'extremely attentive': 1,
         'very busy': 4,
         'too many': 1,
         'super busy': 1,
         'really nice': 1,
         'so excited': 1,
         'too low': 1,
         'very nice': 1,
         'too busy': 2,
         'how busy': 1,
   

In [12]:
# this cell will take some time to run
phrases_dict = [get_list_of_phrases(review['text']) for review in reviews_dict]

we can then get the most common phrases. this is where the fun begins

In [13]:
most_common_phrases = [counter.most_common(5) for counter in phrases_dict]
most_common_phrases

[[('pretty good', 9),
  ('very good', 8),
  ('really good', 7),
  ('very friendly', 4),
  ('very busy', 4)],
 [('so good', 5),
  ('very nice', 4),
  ('really good', 4),
  ('much better', 4),
  ('very fresh', 4)],
 [('pretty good', 6),
  ('really good', 3),
  ('as good', 3),
  ('pretty tasty', 2),
  ('consistently good', 2)],
 [('very clean', 3),
  ('really good', 3),
  ('very good', 2),
  ('anywhere close', 2),
  ('very friendly', 2)],
 [('very good', 5),
  ('very tasty', 5),
  ('really good', 4),
  ('very attentive', 3),
  ('very reasonable', 2)],
 [('pretty good', 4),
  ('really nice', 4),
  ('really good', 3),
  ('too crowded', 3),
  ('nearly empty', 3)],
 [('even worse', 3),
  ('very thorough', 2),
  ('very friendly', 2),
  ('so many', 2),
  ('so much', 2)],
 [('very relaxing', 7),
  ('very professional', 4),
  ('very good', 3),
  ('very pleasant', 3),
  ('absolutely amazing', 3)],
 [('pretty good', 6),
  ('really good', 5),
  ('very good', 4),
  ('so many', 3),
  ('too bad', 3)],


since this does not show any good results as they are not indicative of anything, we look into TF-IDF

In [14]:
# get the most common phrases from all the datasets
all_phrases = dict(sum(phrases_dict, Counter()))
all_phrases

{'always good always good always good ': 15,
 'just typical': 1,
 'just average': 13,
 'also fantastic also fantastic also fantastic also fantastic ': 1,
 'how good': 42,
 'how delicious': 11,
 'very pleased': 46,
 'very good': 592,
 'very friendly': 335,
 'really good': 432,
 'very prompt': 4,
 'real good': 5,
 'too much': 127,
 'pretty decent': 56,
 'pretty good': 414,
 'most recent': 33,
 'pretty average': 8,
 'way friendlier': 1,
 'exceptionally good': 3,
 'much better': 179,
 'extremely greasy': 2,
 'very empty': 2,
 'enough large': 9,
 'barely hot': 2,
 'extremely attentive': 7,
 'very busy': 78,
 'too many': 61,
 'super busy': 28,
 'really nice': 120,
 'so excited': 43,
 'too low': 2,
 'very nice': 361,
 'too busy': 70,
 'how busy': 32,
 'just wonderful': 5,
 'pretty awesome': 16,
 'ridiculously good': 4,
 'just sad': 4,
 'very spacious': 15,
 'all exceptional all exceptional ': 1,
 'very reasonable': 106,
 'very marginal': 1,
 'pretty empty': 13,
 'always friendly always friend

In [15]:
common_phrases_list = [counter.most_common() for counter in phrases_dict]
common_phrases = [tuples for sublist in common_phrases_list for tuples in sublist]
common_phrases

[('pretty good', 9),
 ('very good', 8),
 ('really good', 7),
 ('very friendly', 4),
 ('very busy', 4),
 ('very pleased', 2),
 ('too much', 2),
 ('much better', 2),
 ('too busy', 2),
 ('very spacious', 2),
 ('very large', 2),
 ('very attentive', 2),
 ('super friendly', 2),
 ('so good', 2),
 ('always good always good always good ', 1),
 ('just typical', 1),
 ('just average', 1),
 ('also fantastic also fantastic also fantastic also fantastic ', 1),
 ('how good', 1),
 ('how delicious', 1),
 ('very prompt', 1),
 ('real good', 1),
 ('pretty decent', 1),
 ('most recent', 1),
 ('pretty average', 1),
 ('way friendlier', 1),
 ('exceptionally good', 1),
 ('extremely greasy', 1),
 ('very empty', 1),
 ('enough large', 1),
 ('barely hot', 1),
 ('extremely attentive', 1),
 ('too many', 1),
 ('super busy', 1),
 ('really nice', 1),
 ('so excited', 1),
 ('too low', 1),
 ('very nice', 1),
 ('how busy', 1),
 ('just wonderful', 1),
 ('pretty awesome', 1),
 ('ridiculously good', 1),
 ('just sad', 1),
 ('all

In [16]:
DF = {}
for i, phrase_tuple in enumerate(common_phrases):
    phrase: str = phrase_tuple[0]
    try:
        DF[phrase].add(i)
    except:
        DF[phrase] = {i}

In [17]:
# count across the entire document
for i in DF:
    DF[i] = len(DF[i])
DF

{'pretty good': 120,
 'very good': 134,
 'really good': 127,
 'very friendly': 121,
 'very busy': 47,
 'very pleased': 39,
 'too much': 76,
 'much better': 94,
 'too busy': 50,
 'very spacious': 13,
 'very large': 33,
 'very attentive': 52,
 'super friendly': 79,
 'so good': 108,
 'always good always good always good ': 15,
 'just typical': 1,
 'just average': 12,
 'also fantastic also fantastic also fantastic also fantastic ': 1,
 'how good': 34,
 'how delicious': 10,
 'very prompt': 4,
 'real good': 5,
 'pretty decent': 42,
 'most recent': 27,
 'pretty average': 6,
 'way friendlier': 1,
 'exceptionally good': 3,
 'extremely greasy': 2,
 'very empty': 2,
 'enough large': 9,
 'barely hot': 2,
 'extremely attentive': 7,
 'too many': 47,
 'super busy': 23,
 'really nice': 81,
 'so excited': 28,
 'too low': 2,
 'very nice': 129,
 'how busy': 27,
 'just wonderful': 5,
 'pretty awesome': 16,
 'ridiculously good': 4,
 'just sad': 4,
 'all exceptional all exceptional ': 1,
 'very reasonable':

In [18]:
total_vocab = [x for x in DF]
print(total_vocab)

['pretty good', 'very good', 'really good', 'very friendly', 'very busy', 'very pleased', 'too much', 'much better', 'too busy', 'very spacious', 'very large', 'very attentive', 'super friendly', 'so good', 'always good always good always good ', 'just typical', 'just average', 'also fantastic also fantastic also fantastic also fantastic ', 'how good', 'how delicious', 'very prompt', 'real good', 'pretty decent', 'most recent', 'pretty average', 'way friendlier', 'exceptionally good', 'extremely greasy', 'very empty', 'enough large', 'barely hot', 'extremely attentive', 'too many', 'super busy', 'really nice', 'so excited', 'too low', 'very nice', 'how busy', 'just wonderful', 'pretty awesome', 'ridiculously good', 'just sad', 'all exceptional all exceptional ', 'very reasonable', 'very marginal', 'pretty empty', 'always friendly always friendly ', 'decent(not worth decent(not worth decent(not worth decent(not worth ', 'very disappointing', 'all lumpy', 'very perticular', 'so bizarre',

In [19]:
DocumentTF = []
for i, sublist in enumerate(common_phrases_list):
    TF = defaultdict(int)
    adj_phrase_count: int = 0
    for tuples in sublist:
        adj_phrase_count += tuples[1]
    
    for tuples in sublist:
        TF[tuples[0]] = tuples[1]/adj_phrase_count
    DocumentTF.append(TF)

In [20]:
def computeIDF(word: str) -> float:
    N = len(common_phrases_list)
    num_review = 0
    for reviews in common_phrases_list:
        for review_tuples in reviews:
            if word == review_tuples[0]:
                num_review += 1
    return math.log(N / float(num_review))

In [21]:
temp_idf_dict = defaultdict(int)
for word in total_vocab:
    temp_idf_dict[word] = computeIDF(word)
# idf_list = [{word: computeIDF(word)} for word in total_vocab]

In [22]:
DocumentTF[0].keys()

dict_keys(['pretty good', 'very good', 'really good', 'very friendly', 'very busy', 'very pleased', 'too much', 'much better', 'too busy', 'very spacious', 'very large', 'very attentive', 'super friendly', 'so good', 'always good always good always good ', 'just typical', 'just average', 'also fantastic also fantastic also fantastic also fantastic ', 'how good', 'how delicious', 'very prompt', 'real good', 'pretty decent', 'most recent', 'pretty average', 'way friendlier', 'exceptionally good', 'extremely greasy', 'very empty', 'enough large', 'barely hot', 'extremely attentive', 'too many', 'super busy', 'really nice', 'so excited', 'too low', 'very nice', 'how busy', 'just wonderful', 'pretty awesome', 'ridiculously good', 'just sad', 'all exceptional all exceptional ', 'very reasonable', 'very marginal', 'pretty empty', 'always friendly always friendly ', 'decent(not worth decent(not worth decent(not worth decent(not worth ', 'very disappointing', 'all lumpy', 'very perticular', 'so

In [23]:
DocumentTF[0]['very good']

0.058823529411764705

In [24]:
temp_idf_dict['very good']

0.13259812144152416

In [25]:
def computeTFIDF(TF, idf_list):
    TFIDF = defaultdict(float)
    for phrase in TF.keys():
        phrase_tf_idf = TF[phrase] * idf_list[phrase]
        TFIDF[phrase] = phrase_tf_idf
    return TFIDF

In [26]:
TFIDFList = [computeTFIDF(TF, temp_idf_dict) for TF in DocumentTF]

In [27]:
TFIDFList[0]

defaultdict(float,
            {'pretty good': 0.016077320643334593,
             'very good': 0.007799889496560245,
             'really good': 0.009586440033359631,
             'very friendly': 0.006901393405755718,
             'very busy': 0.03471442116712873,
             'very pleased': 0.020101121695041014,
             'too much': 0.010289773251560358,
             'much better': 0.007163869692976934,
             'too busy': 0.01644727817594543,
             'very spacious': 0.03625718476368969,
             'very large': 0.022557799410675813,
             'very attentive': 0.015870502982514825,
             'super friendly': 0.009720442190079618,
             'so good': 0.005122157268650233,
             'always good always good always good ': 0.017076380296251657,
             'just typical': 0.036988514127885556,
             'just average': 0.018717141702973786,
             'also fantastic also fantastic also fantastic also fantastic ': 0.036988514127885556,
            

In [28]:
max_0 = max(TFIDFList[0], key=TFIDFList[0].get)

In [29]:
max_0

'just typical'

### Cloud tries in jeth.ipynb

In [30]:
import gensim
from gensim import models
from gensim.models import Word2Vec, KeyedVectors

from gensim.models import Phrases
# Train a bigram detector.
bigram_transformer = Phrases(small_business_phrases)
# remove duplicates
common_phrases_list1 = []
for i in common_phrases_list:
    for j in i:
        common_phrases_list1.append(j[0])
common_phrases_list1 = set(common_phrases_list1)
common_phrases_list1 = list(common_phrases_list1)
# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
model = Word2Vec([common_phrases_list1], min_count=1)

In [31]:
import random
random_word = random.choice(model.wv.index_to_key)

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'index_to_key'

In [ ]:
random_word

In [32]:
model.most_similar('more creamy')[:5]

C:\Users\colot\AppData\Local\Temp/ipykernel_18992/1342444753.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('more creamy')[:5]


[('barely warm', 0.3974587917327881),
 ('very so impressed', 0.39026468992233276),
 ('so welcome', 0.3595360815525055),
 ('just worthy', 0.3468708097934723),
 ('good sized', 0.32568883895874023)]

In [33]:
rare_phrase = []
for i in common_phrases_list1:
    j = model.wv.most_similar(i)[0][1]
    if j < 0.30:
        rare_phrase.append(i)
rare_phrase

[]

### claudia_tried, claudia_gave_up
### jeth_continues

In [37]:
X = model[model.wv.vocab]
print(gensim.__version__)

3.8.3


C:\Users\colot\AppData\Local\Temp/ipykernel_18992/2815634701.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  X = model[model.wv.vocab]


In [38]:
from nltk.cluster import KMeansClusterer
import nltk
NUM_CLUSTERS=3
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
print (assigned_clusters)

[0, 1, 1, 0, 2, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 0, 1, 2, 2, 1, 1, 2, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 0, 1, 0, 0, 1, 2, 2, 1, 1, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 0, 2, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 1, 1, 2, 0, 0, 0, 2, 1, 0, 0, 1, 2, 1, 0, 2, 1, 0, 0, 1, 0, 2, 2, 2, 1, 2, 1, 0, 0, 0, 2, 1, 2, 2, 1, 1, 0, 0, 1, 2, 0, 2, 1, 2, 2, 1, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 1, 0, 0, 2, 2, 1, 2, 1, 1, 1, 0, 0, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0, 2, 0, 0, 2, 1, 2, 0, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 1, 1, 0, 2, 2, 1, 2, 1, 2, 0, 0, 1, 1, 2, 0, 2, 1, 2, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 0, 0, 1, 2, 0, 2, 2, 2, 1, 1, 2, 0, 2, 0, 0, 2, 0, 1, 1, 0, 0, 2, 2, 2, 1, 1, 0, 0, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 2, 0, 2, 1, 0, 1, 1, 2, 2, 2, 1, 1, 1, 2, 0, 0, 2, 2, 0, 2, 1, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 1, 0, 2, 

In [39]:
words = list(model.wv.vocab)
for i, word in enumerate(words):  
    print (word + ":" + str(assigned_clusters[i]))

most impressive:0
incredibly odd:1
pretty romantic:1
incredibly awful:0
almost non almost non almost non almost non almost non almost non :2
more visited:1
absolutely solid:1
extremely tough:1
so instant:2
really good:0
so undercooked:1
so lame:1
so worried:1
really cold:2
so cute:1
very classy:2
so average:1
actually gross actually gross :2
very sad:1
all delicious all delicious all delicious :1
always bad always bad always bad :1
most genuine:2
too separate:0
so careful:1
completely horrendous:2
really really disappointed:2
whatsoever pushy:1
also friendly also friendly also friendly also friendly :1
of rough:2
very fast:1
very convenient:0
exceptionally meh:2
that great:0
extremely unwelcome:2
well informed:1
too way spicy:0
around horrible:2
incredibly loud:2
insanely confusing:1
more inconvenient:0
enough hungry:0
quite negative:0
pretty prompt:0
pretty huge:1
far unacceptable:1
particularly authentic:1
so passionate:2
back numerous:2
enough clear:2
terribly inconsistent:2
fiery h

alteringly good:0
always standard always standard always standard always standard :0
primarily turkish:1
inside spacious:2
fairly slow:1
thankfully so comfortable:1
so hard:1
too soggy:0
amazingly authentic:0
how floored:1
just glad:1
more painful:2
how stuffy:2
as just gorgeous:2
quite declious:1
very successful:0
really concerned:2
more suited:2
super just small:2
so filthy:0
just tired:2
ediblely attractive:2
too underdressed:0
quite fast:0
overall ok:0
barely just underbaked:2
all interested:2
politically correct:2
exceptionally nice:2
enough gracious:1
much weekly:2
how amazing:2
entirely crispy:0
super cool:0
pretty worried:0
blatantly direct:0
enough drunk:1
so damn:0
all skimpy:1
just bad:1
dry dry:0
how average:0
wrenchingly painful:2
really large:0
incredibly creative:2
as painful:0
more different:1
then afghan:0
more sophisticated:2
how elegant:0
too little:0
embarrassingly slow:2
completely dry:0
still warm:0
very scarce:1
always willing always willing :2
truly amazing:1
ex

very familiar:0
so focused:2
way overcooked:0
much ok:2
more kind:2
that hot:2
really disgusting:0
so enormous:1
too back bad:2
too wide----wider:1
vaguely similar:1
pretty spotty:2
really tender:2
pretty unhappy:1
legitimately spicy:1
as moist:0
as delicious:2
very personal:2
kinda dissapointed:1
especially friendly:1
how clean:2
extremely especially rude:2
so dark:2
how crazy:1
always busy always busy always busy :0
shockingly small:2
very funny:2
more fresh:0
most opulent:2
too drunk:1
so tasteless:1
magically so pungent:2
even clean:2
enough varied:1
normally tight:0
relatively good:0
quite satisfying:2
always prompt always prompt :0
specifically shish:0
incredibly basic:1
very aggressive:0
too bothersome:0
really welcome:1
highly satisfied:1
pretty bare:0
so flavorful:2
enough lucky:0
extremely quaint:2
definitely worth definitely worth definitely worth :0
very dessert:1
pretty large:0
that pricey:1
pretty excited:2
slightly lacking:0
all fresh all fresh :0
too overpriced:2
most n

extremely welcoming:2
always weird always weird :0
always amazing always amazing always amazing always amazing always amazing :2
very bitter:1
fairly average:1
clearly irritated:1
truly great:0
very wish:0
deceivingly big:2
too crammed:0
as appealing:2
either definitely good:0
always sweet always sweet always sweet always sweet always sweet :2
always wet always wet :2
so just bad:1
very disappointing:2
pretty aggressive:2
how drunk:2
pretty yummy:0
most upset:1
always hot:2
extremely accomodating:0
very bready:0
so hidden:2
more straightforward:0
very forthcoming:1
really neat:1
so back many:0
rather warm:2
ever popular:2
very talented:1
just happy:0
much thinner:2
very complex:0
quite fascinated:1
only friendly:2
fully comfortable:0
so respectful:0
very disorganized-:0
super simple:1
absolutely tender:0
more considerably expensive:2
infinitely better:2
as much:1
too cute:2
more iced:2
super high:2
extremely comfortable:2
perfect friendly:1
very often crowded:0
very so busy:1
equally p

incredibly sensitive:0
almost immediate:1
essentially unlimited:1
almost overwhelming:2
options- traditional:1
perfectly ripe:1
really just good:1
super addictive:1
again disappointed:0
also fantastic also fantastic also fantastic :1
that just strong:1
consistenly awesome:1
very lame:2
definitely healthy:0
more better accurate:0
rather disappointed:1
super unfriendly:0
totally relaxing:2
more positive:2
more perfect:0
of panicked:2
here hot:1
very tolerant:0
far great:0
most polite:2
too obvious:1
actually bad actually bad :2
just average:1
also dynamite also dynamite also dynamite also dynamite :2
reallllly gross:2
beautifully crisp:1
both hot:0
so quite many:0
ever first:0
pretty simple:2
super organized:1
super professional:1
as fresh:0
even good:1
so crisp:2
pretty back sure:1
otherwise fine:0
seriously desperate:1
very optimistic:0
very frustrated:2
pretty cheesy:1
so inept:1
quite convincing:0
ever correct ever correct :2
so just many:2
very tasteless:0
so spoiled:1
slightly anno